In [1]:
%load_ext autoreload
%autoreload 2
from econml._ortho_learner import _crossfit

In [7]:
from econml.dml import LinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
import numpy as np
X = np.random.normal(size=(100000, 3))
y = X[:, 0] + np.random.normal(size=(100000,))
est = LinearDMLCateEstimator(model_y=LinearRegression(), model_t=LinearRegression())
est.fit(y, X[:, 0], X[:, [1]], X[:, 2:], inference='statsmodels')
print(X[:1])
print(est.effect(X[:1, [1]]))
print(est.effect_interval(X[:1, [1]]))
print(est.coef_)
print(est.coef__interval())
print(est.const_marginal_effect(X[:1, [1]]))

[[ 1.30061979  1.91105845 -0.46179321]]
[1.00171855]
(array([0.99046797]), array([1.01296913]))
[0.99920984 0.00131274]
(array([ 0.99401435, -0.00390604]), array([1.00440533, 0.00653151]))
[1.00171855]


In [25]:
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, MultiTaskLassoCV
import numpy as np
X = np.random.normal(size=(1000, 10))
y = X[:, 0] + np.random.normal(size=(1000,))
est = SparseLinearDMLCateEstimator(model_y=MultiTaskLassoCV(cv=3), model_t=LassoCV(cv=3),
                                   model_final=MultiTaskLassoCV(cv=3))
dx = 8
est.fit(np.hstack([y.reshape(-1,1), y.reshape(-1,1)]), X[:, 0], X[:, 1:dx], X[:, dx:])
print(X[:1])
print(est.effect(X[:1, 1:dx]))
print(est.model_final.coef_)
print(est.const_marginal_effect(X[:1, 1:dx]))

[[-0.68287838 -0.80819115 -0.18035347  0.91400753  0.35464864  0.02063007
  -2.98266229  0.97470114  1.21184005  0.46500733]]


/Users/vasilis/Documents/EconML/econml/dml.py:160: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  UserWarning)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()